In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
try:
    print_log
except: 
    print_log = False

In [ ]:
model_key = metys.g_prj.models.current 
data_path = os.path.join(metys.g_prj.folder, '%s_area_sources.pkl' % model_key)
fin = open(data_path,'rb') 
sources = pickle.load(fin)
fin.close()

In [ ]:
from openquake.hazardlib.mfd import TruncatedGRMFD

bin_width = float(metys.g_prj.mod[model_key]['mfd_binwidth'])
min_mag = float(metys.g_prj.mod[model_key]['mfd_min_magnitude'])

sc = sources[area_source_ids_list[0]]
src.mfd = TruncatedGRMFD(min_mag = min_mag,
                         max_mag = src.m_max,
                         bin_width = bin_width,
                         a_val = src.a_gr,
                         b_val = src.b_gr)